Descargar el dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ai21ds06anilriswal/terrain-dataset")

print("Path to dataset files:", path)

100%|██████████| 239M/239M [00:03<00:00, 78.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ai21ds06anilriswal/terrain-dataset/versions/1


Para Comprobar Imagenes con la IA entrenada

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import tensorflow as tf # Import tensorflow

# Carga una imagen de prueba (que el modelo NUNCA haya visto)
# Replace 'G6572_f_1.jpg' with the actual path to your new image file
img_path = '/root/.cache/kagglehub/datasets/ai21ds06anilriswal/terrain-dataset/test/G6572_f_1.jpg'
img = image.load_img(img_path, target_size=IMG_SIZE)

# Convierte la imagen a números
img_array = image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Crea un "batch" de 1 imagen

# ¡Predecir!
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

# Obtener los nombres de las clases (los nombres de tus carpetas)
class_names = train_dataset.class_names

# Imprimir el resultado
print(
    "Esta imagen es un {} con un {:.2f}% de confianza."
    .format(class_names[np.argmax(score)], 100 * tf.reduce_max(score)) # Use tf.reduce_max for consistency
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
Esta imagen es un Rocky_Terrain con un 40.37% de confianza.


Guardar

In [ ]:
# Save the entire model in the SavedModel format for deployment to Google Drive.
# Asegúrate de que Google Drive esté montado usando la celda anterior.
drive_path = '/content/drive/My Drive/my_terrain_model.keras'
model.export(drive_path)
print(f"Model saved successfully to {drive_path}!")

Saved artifact at '/content/drive/My Drive/my_terrain_model.keras'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name='keras_tensor_634')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  139309947240080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139309947238544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139309947239888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139309947240272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139309924614416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139309924614800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139309924614992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139309924615568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139309924614608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139309924616336: TensorSpec(shape=(), dtype=tf.res

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Entrenamiento

In [ ]:
import tensorflow as tf
import os

# Definir el tamaño de las imágenes (todas deben ser iguales)
IMG_SIZE = (180, 180)
BATCH_SIZE = 32 # Cuántas imágenes ve el modelo a la vez

# --- CAMBIO 1: Ruta correcta ---
# Esta es la ruta que contiene las carpetas de clases (Grassy_Terrain, Marshy_Terrain, etc.)
data_dir = "/root/.cache/kagglehub/datasets/ai21ds06anilriswal/terrain-dataset/versions/1/terrain dataset"

# --- CAMBIO 2: Crear 'train_dataset' con división de validación y filtrado ---
# Usamos validation_split=0.2 para reservar el 20% de los datos para validación.
# Usamos subset='training' para decirle que este es el 80% de entrenamiento.
# Usamos seed=42 para asegurar que la división sea siempre la misma y no haya solapamiento.
train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=42, # Importante para que no se solapen los datos de ent. y val.
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

# --- CAMBIO 3: Crear 'validation_dataset' desde la MISMA división con filtrado---
# Apuntamos al MISMO directorio, pero ahora pedimos el subconjunto 'validation'.
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=42, # DEBE ser la misma 'seed' que antes
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

# --- CAMBIO 4: Obtener el número de clases automáticamente ---
# Contamos cuántas carpetas (clases) hay en el directorio
num_classes = len(train_dataset.class_names)
print(f"Número de clases encontradas: {num_classes}")
print(f"Nombres de las clases: {train_dataset.class_names}")


# 1. Cargar el modelo base (experto) sin su "cabeza" (la parte final que clasifica)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), # Use IMG_SIZE here
    include_top=False,        # ¡No incluyas la cabeza original!
    weights='imagenet'        # Cargar los "conocimientos" pre-entrenados
)

# 2. Congelar al experto para no arruinar su conocimiento
base_model.trainable = False

# 3. Construir tu modelo: pones el experto y añades tu propia "cabeza"
model = tf.keras.Sequential([
    # Normalizar pixeles de 0-255 a 0-1 (mejora el aprendizaje)
    tf.keras.layers.Rescaling(1./255),

    # El experto
    base_model,

    # Una capa para "aplanar" la salida del experto
    tf.keras.layers.GlobalAveragePooling2D(),

    # Una capa de "filtro" para evitar que se aprenda de memoria
    tf.keras.layers.Dropout(0.2),

    # --- CAMBIO 5: Usar el número de clases dinámico ---
    # La capa final: una neurona por cada clase que tengas
    tf.keras.layers.Dense(num_classes, activation='softmax') # Softmax da probabilidades
])

# 1. Compilar: Definir el optimizador, la función de error y la métrica
model.compile(
    optimizer='adam', # 'adam' es el optimizador estándar, funciona muy bien
    loss=tf.keras.losses.SparseCategoricalCrossentropy(), # Función para clasificación
    metrics=['accuracy'] # Queremos ver la precisión
)

# 2. Entrenar: ¡Aquí ocurre la magia!
EPOCHS = 10 # Cuántas veces el modelo va a "repasar" todos los datos

history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=validation_dataset # Ahora sí tenemos datos de validación
)

print("¡Entrenamiento completado!")

Found 1894 files belonging to 5 classes.
Using 1516 files for training.
Found 1894 files belonging to 5 classes.
Using 378 files for validation.
Número de clases encontradas: 5
Nombres de las clases: ['Grassy_Terrain', 'Marshy_Terrain', 'Other_Image', 'Rocky_Terrain', 'Sandy_Terrain']


/tmp/ipython-input-559840936.py:44: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(


Epoch 1/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 122s 2s/step - accuracy: 0.4225 - loss: 1.4669 - val_accuracy: 0.8757 - val_loss: 0.4052
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - accuracy: 0.8683 - loss: 0.3941 - val_accuracy: 0.9048 - val_loss: 0.3039
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step - accuracy: 0.8960 - loss: 0.3119 - val_accuracy: 0.9312 - val_loss: 0.2606
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 128s 3s/step - accuracy: 0.9211 - loss: 0.2546 - val_accuracy: 0.9392 - val_loss: 0.2381
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 107s 2s/step - accuracy: 0.9441 - loss: 0.1977 - val_accuracy: 0.9365 - val_loss: 0.2260
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - accuracy: 0.9365 - loss: 0.1963 - val_accuracy: 0.9471 - val_loss: 0.2062
Epoch 7/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - accuracy: 0.9450 - loss: 0.1616 - val_accuracy: 0.9471 - val_loss: 0.2129
Epoch 8/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - accuracy: 0.9532 - loss: 0.1506 - val_accuracy: 0.9418 - v

Elimina archivos problematicos

In [ ]:
import tensorflow as tf
from pathlib import Path
import os

# Asegúrate de que esta es la ruta correcta a tu carpeta de clases
data_dir = Path("/root/.cache/kagglehub/datasets/ai21ds06anilriswal/terrain-dataset/versions/1/terrain dataset")

print("Buscando y eliminando archivos corruptos o que no sean imágenes...")

# .rglob('*') busca en todas las subcarpetas
for filepath in data_dir.rglob('*'):
    # Nos saltamos las carpetas, solo queremos archivos
    if filepath.is_file():
        try:
            # 1. Leer el archivo
            img_bytes = tf.io.read_file(str(filepath))
            # 2. Intentar decodificarlo como imagen
            # Si esto falla, saltará la excepción
            tf.io.decode_image(img_bytes)
        except tf.errors.InvalidArgumentError:
            # ¡Encontramos al culpable!
            print(f"--> Eliminando archivo problemático: {filepath}")
            os.remove(filepath)
        except Exception as e:
            # Otro error (ej. permisos)
            print(f"--> NO SE PUDO LEER {filepath}: {e}")

print("Búsqueda y eliminación finalizadas.")

Buscando y eliminando archivos corruptos o que no sean imágenes...
--> Eliminando archivo problemático: /root/.cache/kagglehub/datasets/ai21ds06anilriswal/terrain-dataset/versions/1/terrain dataset/Rocky_Terrain/s (128).jpg
--> Eliminando archivo problemático: /root/.cache/kagglehub/datasets/ai21ds06anilriswal/terrain-dataset/versions/1/terrain dataset/Rocky_Terrain/s (491).jpg
--> Eliminando archivo problemático: /root/.cache/kagglehub/datasets/ai21ds06anilriswal/terrain-dataset/versions/1/terrain dataset/Rocky_Terrain/s (551).jpg
--> Eliminando archivo problemático: /root/.cache/kagglehub/datasets/ai21ds06anilriswal/terrain-dataset/versions/1/terrain dataset/Rocky_Terrain/s (484).jpg
--> Eliminando archivo problemático: /root/.cache/kagglehub/datasets/ai21ds06anilriswal/terrain-dataset/versions/1/terrain dataset/Rocky_Terrain/s (488).jpg
--> Eliminando archivo problemático: /root/.cache/kagglehub/datasets/ai21ds06anilriswal/terrain-dataset/versions/1/terrain dataset/Rocky_Terrain/s (